# Reconcile instruments

You can run the cells below directly in LUSID's JupyterHub.

The `%%luminesce` is a magic command which passes the cell query string to Lumipy,
which then returns a DataFrame.
    

#### Step 1: Create instrument properties

In [ ]:
%%luminesce

-- ==================================================================
-- Description:
-- 1. In this example, we create some custom properties in LUSID,
-- loaded from an Excel file.
-- ===================================================================

-- Load a CSV file of properties from LUSID Drive

@properties = use Drive.Excel
--file=/luminesce-examples/fi_instrument_master.xlsx
--worksheet=properties
enduse;

-- Transform the CSV data into a LUSID format

@property_definition = select
PropertyCode as [DisplayName],
'Instrument' as [Domain],
Scope as [PropertyScope],
PropertyCode as [PropertyCode],
'Property' as [ConstraintStyle],
'system' as [DataTypeScope],
DataType as [DataTypeCode]
from @properties;

-- Upload the properties into LUSID

select * from Lusid.Property.Definition.Writer
where ToWrite = @property_definition;



#### Step 2: Setup instrument with properties

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this file, we load instruments into LUSID with the new
-- properties defined in the first file.
-- ===============================================================

@instruments_source_a = use Drive.Excel
--file=/luminesce-examples/fi_instrument_master.xlsx
--worksheet=SourceA
enduse;

-- 1. Upload instruments with default properties to Lusid
-- Transform data using Luminesce
@fi_instruments = select
Name as DisplayName,
Currency as DomCcy,
'Bonds' as SimpleInstrumentType,
'Credit' as AssetClass,
ClientInternal,
CountryIssue,
IssuePrice
from @instruments_source_a;

-- Upload the transformed data into LUSID
@write_instruments =
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @fi_instruments
and DeletePropertiesWhereNull=True;

-- 2. Upload values for custom properties to Lusid
-- Transform data using Luminesce
@custom_props =
select ClientInternal, CountryIssue, HoldingType, InstrumentType, GICSLevel1,
GICSLevel2, SnPRating, MoodyRating, MIFIDFlag
from @instruments_source_a;

@unpivoted =
use Tools.Unpivot with @custom_props
--key=ClientInternal
--keyIsNotUnique
enduse;

@instr_props =
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain,
    a.ValueColumnName as PropertyCode, 'SourceA' as PropertyScope, a.ValueText as Value
from @unpivoted a
inner join Lusid.Instrument li
   on li.ClientInternal = a.ClientInternal;

-- Upload the transformed data into LUSID
@write_props =
select *
from Lusid.Property.Writer
where ToWrite = @instr_props;



#### Step 3: Load instruments into source a

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this file, we load instruments into LUSID with the new
-- properties defined in the first file.
-- ===============================================================

@instruments_source_a = use Drive.Excel
--file=/luminesce-examples/fi_instrument_master.xlsx
--worksheet=SourceA
enduse;

-- 1. Upload instruments with default properties to Lusid
-- Transform data using Luminesce
@fi_instruments = select
Name as DisplayName,
Currency as DomCcy,
'Bonds' as SimpleInstrumentType,
'Credit' as AssetClass,
ClientInternal,
CountryIssue,
IssuePrice
from @instruments_source_a;

-- Upload the transformed data into LUSID
@write_instruments =
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @fi_instruments
and DeletePropertiesWhereNull=True;

-- 2. Upload values for custom properties to Lusid
-- Transform data using Luminesce
@custom_props =
select ClientInternal, HoldingType, InstrumentType, GICSLevel1,
GICSLevel2, SnPRating, MoodyRating, MIFIDFlag
from @instruments_source_a;

@unpivoted =
use Tools.Unpivot with @custom_props
--key=ClientInternal
--keyIsNotUnique
enduse;

@instr_props =
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain,
    a.ValueColumnName as PropertyCode, 'SourceA' as PropertyScope, a.ValueText as Value
from @unpivoted a
inner join Lusid.Instrument li
   on li.ClientInternal = a.ClientInternal;

-- Upload the transformed data into LUSID
@write_props =
select *
from Lusid.Property.Writer
where ToWrite = @instr_props;



#### Step 4: Load instruments into source b

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this file, we load instruments into LUSID with the new
-- properties defined in the first file.
-- ===============================================================

@instruments_source_b =
use Drive.Excel
--file=/luminesce-examples/fi_instrument_master.xlsx
--worksheet=SourceB
enduse;

-- 1. Upload instruments with default properties to Lusid
-- Transform data using Luminesce
@fi_instruments =
select Name as DisplayName, Currency as DomCcy, 'Bonds' as SimpleInstrumentType, 'Credit' as AssetClass,
   client_internal as ClientInternal
from @instruments_source_b;

-- Upload the transformed data into LUSID
@write_instruments =
select *
from Lusid.Instrument.SimpleInstrument.Writer
where ToWrite = @fi_instruments
   and DeletePropertiesWhereNull = True;

-- 2. Upload values for custom properties to Lusid
-- Transform data using Luminesce
@custom_props =
select client_internal, holding_type, instrument_type, gics_level_1, gics_level_2, snp_rating, moody_rating,
   mifid_flag, country_issue, issue_price
from @instruments_source_b;

@unpivoted =
use Tools.Unpivot with @custom_props
--key=client_internal
--keyIsNotUnique
enduse;

@instr_props =
select li.LusidInstrumentId as EntityId, 'LusidInstrumentId' as EntityIdType, 'Instrument' as Domain, 'SourceB' as
   PropertyScope, a.ValueColumnName as PropertyCode, a.ValueText as Value
from @unpivoted a
inner join Lusid.Instrument li
   on li.ClientInternal = a.client_internal;

-- Upload the transformed data into LUSID
@write_props =
select *
from Lusid.Property.Writer
where ToWrite = @instr_props;



#### Step 5: Run reconciliation

In [ ]:
%%luminesce

-- ===============================================================
-- Description:
-- In this file, we run a reconciliation on three of the properties
-- ===============================================================

-- 1. Create a view of instruments with custom properties
-- Get data from LUSID
@instr_props =
select li.ClientInternal, li.DisplayName, PropertyCode, Value
from Lusid.Instrument.Property p
inner join Lusid.Instrument.SimpleInstrument li
   on li.LusidInstrumentId = p.InstrumentId
where li.SimpleInstrumentType = 'Bonds'
   and li.AssetClass = 'Credit'
    and ((p.propertyscope  = 'SourceA'
    and p.propertycode in ('HoldingType', 'CountryIssue', 'GICSLevel1', 'InstrumentType'))
    or 
    (p.propertyscope  = 'SourceB'
    and p.propertycode in ('holding_type', 'country_issue', 'gics_level_1', 'instrument_type'))) ;


-- Transform data using luminesce
@pivoted =
use Tools.Pivot with @instr_props
--key=PropertyCode
--aggregateColumns=Value
enduse;

-- 2. Run reconcilliation
select ClientInternal, DisplayName, case
      when HoldingType = holding_type
         then 'Reconciled: ' || HoldingType
      else 'Break: ' || HoldingType || ' versus ' || holding_type
      end as HoldingType, case
      when CountryIssue = country_issue
         then 'Reconciled: ' || CountryIssue
      else 'Break: ' || CountryIssue || ' versus ' || country_issue
      end as CountryIssue, case
      when GICSLevel1 = gics_level_1
         then 'Reconciled: ' || GICSLevel1
      else 'Break: ' || GICSLevel1 || ' versus ' || gics_level_1
      end as GICSLevel1
from @pivoted
where (
      HoldingType is not null
      and InstrumentType is not null
      );

